In [2]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

In [ ]:
def detailinforequest():
    # URL của API
    url = 'https://unipass.customs.go.kr/clip/prlstclsfsrch/retrievePrlstClsfCaseDtl.do'

    # Headers
    headers = {
        'Accept': 'text/html, */*; q=0.01',
        'Accept-Language': 'en-US,en;q=0.9,vi;q=0.8,vi-VN;q=0.7',
        'Connection': 'keep-alive',
        'Content-Type': 'application/x-www-form-urlencoded; charset=UTF-8',
        'Cookie': 'WMONID=N_4A3dINH0I; JSESSIONID=0001O2y8S-yyQQ_lga-knblPV6Lb2QN6662ivRFJ_jK2FPRlLOTKNEsnuhZe_QxBEKq-8U03_kah8EgUgb-_Exa4dmevvhJHEKeNuIq9prRxpJht6ugFASENgz3p_AdMPw4_:eul21',
        'DNT': '1',
        'Origin': 'https://unipass.customs.go.kr',
        'Referer': 'https://unipass.customs.go.kr/clip/index.do',
        'Sec-Fetch-Dest': 'empty',
        'Sec-Fetch-Mode': 'cors',
        'Sec-Fetch-Site': 'same-origin',
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/129.0.0.0 Safari/537.36',
        'X-Requested-With': 'XMLHttpRequest',
        'isAjax': 'true',
        'sec-ch-ua': '"Google Chrome";v="129", "Not=A?Brand";v="8", "Chromium";v="129"',
        'sec-ch-ua-mobile': '?0',
        'sec-ch-ua-platform': '"Windows"'
    }

    # Dữ liệu trong body của yêu cầu
    payload = {
        'cntyCd': 'VN',
        'lngTpcd': 'eng',
        'compareCrrspndNation': '',
        'sctYear': '20240101',
        'hstdYear': '20240101',
        'hsfdYear': '',
        'manlOrgnTpcd': '01',
        'aplyYy': '2024',
        'tabTpcd': '3',
        'hsGnprSrno': '1',
        'searchVal': '0103',
        'euTp': '',
        'tmpManlOrgnTpcd': ''
    }


    # Gửi yêu cầu POST với headers và dữ liệu body
    response = requests.post(url, headers=headers, data=payload,timeout=60)

    # Kiểm tra phản hồi từ API
    if response.status_code == 200:
        return response.text
    else:
        print(f"Row {"index"} Lỗi")
        return None

In [ ]:
def extract_data_from_html(html_content):
    soup = BeautifulSoup(html_content, 'html.parser')
    result = {}
    for table_type in ('org','eng','kor'):
        table = soup.find("table", class_=table_type)
        # Tìm thẻ <p> chứa từ khóa "상세결과 항목 :"
        all_ths = table.findAll('th')
        for th in all_ths: 
            if th.get_text() == "관련 이미지":
                image_elements = table.find('th', text='관련 이미지').find_next_sibling('td').find_all('img')
                image_links = [img['src'] for img in image_elements]
                result[f"{th.get_text()}_{table_type}"] = image_links
            # print(f"Đang xử lý: {catalog}")
            else: 
                elems_html_info = table.find('th', text=th.get_text()).find_next_sibling('td')
                for elem in elems_html_info:
                    if elem.name == 'a':  # Thay đổi thẻ <a> thành văn bản
                        elem.replace_with(elem.text)
                    elif elem.name == 'br':  # Giữ lại thẻ <br>
                        continue
                elem_text = elems_html_info.decode_contents().replace('\r', '').replace('\t', '').replace('\n', '').strip()
                result[f"{th.get_text()}_{table_type}"] = elem_text
    print(result)
    return result